In [1]:
# sage documentation for SQLite interface
# http://doc.sagemath.org/html/en/reference/misc/sage/databases/sql_db.html

In [100]:
from tabulate import tabulate # for making texable tables of the data
import numpy as np #...just because i'm being lazy and using this to take a transpose
import time
load("kraus_tables.sage")
kraus = kraus_table_Q2
DB = CremonaDatabase(); DB
p = 2
# note: the quadratic extensions of Q2 are Q2(sqrt(d)) with d = 3,5,7,2,6,10,14.

objfile = os.path.join('./', '2-ktype_mtype_data.sobj')
label_type_tuples = load(objfile)  # format: (cremona label (string), Kodaira type (Sage custom type), integer)

objfile = os.path.join('./', '2-ktype_mtype_first_examples.sobj')
first_examples = load(objfile)

In [101]:
DB.largest_conductor()

399998

In [3]:
def ctame(S):
    if S == KodairaSymbol('In*'):
        return 1/2
    elif S == KodairaSymbol('II'):
        return 1/6
    elif S == KodairaSymbol('II*'):
        return 5/6
    elif S == KodairaSymbol('III'):
        return 1/4
    elif S == KodairaSymbol('III*'):
        return 3/4
    elif S == KodairaSymbol('IV'):
        return 1/3
    elif S == KodairaSymbol('IV*'):
        return 2/3
    else:
        raise ValueError("Kodaira type In not implemented")

In [4]:
D = SQLDatabase(filename='2-totally-wild.db')

In [268]:
## don't run this unless you have recomputed everything 

##create database:
#from collections import OrderedDict
#table_skeleton = OrderedDict([
#    ('label',{'sql':'TEXT', 'index':True, 'primary_key':True}), # the Cremona label
#    ('mtype',{'sql':'TEXT', 'index':True,}), # monodromy type                         
#    ('ktype',{'sql':'TEXT', 'index':True,}), # Kodaira type
#    ('c',{'sql':'TEXT', 'index':True,}), # the Chai conductor
#    ('c_tame',{'sql':'TEXT', 'index':True,}), # the tame part of the chai conductor
#    ('c_wild',{'sql':'TEXT', 'index':True,}),# the wild part of the chai conductor
#])
#D = SQLDatabase()
#D.create_table('twpg', table_skeleton) # twpg = totally wild potentially good
## D.show('twpg')

#t0 = time.time()
#for (label, symbol, mtype) in label_type_tuples:
#    if mtype in [2,4,8]:
#        E = EllipticCurve(label)
#        LD = E.local_data(2)
#
#        c = LD.discriminant_valuation()/12       
#        ct = ctame(symbol)
#        cw = c - ct
#        
#        D.add_row('twpg', (label, mtype, symbol, c, ct, cw))
#        # print((label, c, ct, cw, mtype))
## D.show('twpg')
#t1 = time.time()
#print('Time: {}'.format(t1-t0))
## 194.962843895s on my personal laptop on 5 Oct. 2019

# D.save('/home/cody/Dropbox/sage-notebooks/unstable-ECs/2-totally-wild.db')

Time: 191.098484039


In [5]:
# which chai conductors occur?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['c'], 
                 'expression':['c','>','0']})
Qout = Q.query_results(); 
Qout = set([str(l[0]) for l in Qout]); Qout

{'1', '1/2', '3/2', '3/4', '5/4'}

In [89]:
# function for making latex table from the data below
def tablify_query(Q):
    Q = set(Q.query_results());
    Q = [[KodairaSymbol(str(t[0]))]+map(Rational, t[1:]) for t in Q]
    Q = [['Kodaira type', 'c(E)', 'c_tame(E)', 'c_wild(E)']] + Q
    Q = np.array(Q).T.tolist() # I use numpy for this, ha
    print(tabulate(Q))
    return(latex(tabulate(Q)))

In [96]:
# conductors and types when M = C_2?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','2']})
tablify_query(Q)

------------  ---  ---  ---  ---
Kodaira type  II   II*  I4*  I8*
c(E)          1/2  1    1    3/2
c_tame(E)     1/6  5/6  1/2  1/2
c_wild(E)     1/3  1/6  1/2  1
------------  ---  ---  ---  ---


\begin{array}{l}
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}}}\\
\text{\texttt{Kodaira{ }type{ }{ }II{ }{ }{ }II*{ }{ }I4*{ }{ }I8*}}\\
\text{\texttt{c(E){ }{ }{ }{ }{ }{ }{ }{ }{ }{ }1/2{ }{ }1{ }{ }{ }{ }1{ }{ }{ }{ }3/2}}\\
\text{\texttt{c{\char`\_}tame(E){ }{ }{ }{ }{ }1/6{ }{ }5/6{ }{ }1/2{ }{ }1/2}}\\
\text{\texttt{c{\char`\_}wild(E){ }{ }{ }{ }{ }1/3{ }{ }1/6{ }{ }1/2{ }{ }1}}\\
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}}}
\end{array}

In [97]:
# conductors and types when M = C_4?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','4']})
tablify_query(Q)

------------  ----  ---
Kodaira type  III*  III
c(E)          5/4   3/4
c_tame(E)     3/4   1/4
c_wild(E)     1/2   1/2
------------  ----  ---


\begin{array}{l}
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}\\
\text{\texttt{Kodaira{ }type{ }{ }III*{ }{ }III}}\\
\text{\texttt{c(E){ }{ }{ }{ }{ }{ }{ }{ }{ }{ }5/4{ }{ }{ }3/4}}\\
\text{\texttt{c{\char`\_}tame(E){ }{ }{ }{ }{ }3/4{ }{ }{ }1/4}}\\
\text{\texttt{c{\char`\_}wild(E){ }{ }{ }{ }{ }1/2{ }{ }{ }1/2}}\\
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}
\end{array}

In [98]:
# conductors and types when M = Q_8?
Q = SQLQuery(D, {'table_name':'twpg',
                 'display_cols':['ktype','c', 'c_tame', 'c_wild'], 
                 'expression':['mtype','=','8']})
tablify_query(Q)

------------  ---  ----  ---  ---  ---  ---  ---  ----  ---
Kodaira type  I0*  III*  III  I3*  I2*  III  II   III*  I5*
c(E)          3/4  5/4   3/4  1    1    1/2  1/2  1     5/4
c_tame(E)     1/2  3/4   1/4  1/2  1/2  1/4  1/6  3/4   1/2
c_wild(E)     1/4  1/2   1/2  1/2  1/2  1/4  1/3  1/4   3/4
------------  ---  ----  ---  ---  ---  ---  ---  ----  ---


\begin{array}{l}
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}\\
\text{\texttt{Kodaira{ }type{ }{ }I0*{ }{ }III*{ }{ }III{ }{ }I3*{ }{ }I2*{ }{ }III{ }{ }II{ }{ }{ }III*{ }{ }I5*}}\\
\text{\texttt{c(E){ }{ }{ }{ }{ }{ }{ }{ }{ }{ }3/4{ }{ }5/4{ }{ }{ }3/4{ }{ }1{ }{ }{ }{ }1{ }{ }{ }{ }1/2{ }{ }1/2{ }{ }1{ }{ }{ }{ }{ }5/4}}\\
\text{\texttt{c{\char`\_}tame(E){ }{ }{ }{ }{ }1/2{ }{ }3/4{ }{ }{ }1/4{ }{ }1/2{ }{ }1/2{ }{ }1/4{ }{ }1/6{ }{ }3/4{ }{ }{ }1/2}}\\
\text{\texttt{c{\char`\_}wild(E){ }{ }{ }{ }{ }1/4{ }{ }1/2{ }{ }{ }1/2{ }{ }1/2{ }{ }1/2{ }{ }1/4{ }{ }1/3{ }{ }1/4{ }{ }{ }3/4}}\\
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}
\end{array}

------------  ---  ----  ---  ---  ---  ---  ---  ----  ---
Kodaira type  I0*  III*  III  I3*  I2*  III  II   III*  I5*
c(E)          3/4  5/4   3/4  1    1    1/2  1/2  1     5/4
c_tame(E)     1/2  3/4   1/4  1/2  1/2  1/4  1/6  3/4   1/2
c_wild(E)     1/4  1/2   1/2  1/2  1/2  1/4  1/3  1/4   3/4
------------  ---  ----  ---  ---  ---  ---  ---  ----  ---


\begin{array}{l}
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}\\
\text{\texttt{Kodaira{ }type{ }{ }I0*{ }{ }III*{ }{ }III{ }{ }I3*{ }{ }I2*{ }{ }III{ }{ }II{ }{ }{ }III*{ }{ }I5*}}\\
\text{\texttt{c(E){ }{ }{ }{ }{ }{ }{ }{ }{ }{ }3/4{ }{ }5/4{ }{ }{ }3/4{ }{ }1{ }{ }{ }{ }1{ }{ }{ }{ }1/2{ }{ }1/2{ }{ }1{ }{ }{ }{ }{ }5/4}}\\
\text{\texttt{c{\char`\_}tame(E){ }{ }{ }{ }{ }1/2{ }{ }3/4{ }{ }{ }1/4{ }{ }1/2{ }{ }1/2{ }{ }1/4{ }{ }1/6{ }{ }3/4{ }{ }{ }1/2}}\\
\text{\texttt{c{\char`\_}wild(E){ }{ }{ }{ }{ }1/4{ }{ }1/2{ }{ }{ }1/2{ }{ }1/2{ }{ }1/2{ }{ }1/4{ }{ }1/3{ }{ }1/4{ }{ }{ }3/4}}\\
\text{\texttt{{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{ }{ }{-}{-}{-}{-}{ }{ }{-}{-}{-}}}
\end{array}